## Setup

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [2]:
import pandas as pd

df = pd.read_csv("hf://datasets/TungHamHoc/sentiment-mental-health/sentiment-mental-health.csv")

# Tentukan path folder 'data' di direktori saat ini
save_dir = "data"
os.makedirs(save_dir, exist_ok=True)  # Buat folder jika belum ada

# Simpan file CSV dalam folder 'data'
save_path = os.path.join(save_dir, "data.csv")
df.to_csv(save_path, index=False)

print(f"File disimpan di {save_path}")

File disimpan di data\data.csv


In [3]:
PIPELINE_NAME = "heart-attack-pipeline"
SCHEMA_PIPELINE_NAME = "heart-att-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where createdmodels from the pipeline will be exported
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root = PIPELINE_ROOT)

## Data Ingestion

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 6
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation
### 1. Summary Statistic

In [7]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs['examples']
)

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 7
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

### 2. Data Schema

In [9]:
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 8
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'statement',BYTES,required,,-
'status',STRING,required,,'status'


,Values
Domain,
'status',"'Anxiety', 'Depression', 'Normal', 'Suicidal'"


### 3. Identifikasi Anomali

In [11]:
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs["statistics"],
    schema = schema_gen.outputs["schema"]
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 9
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs["anomalies"])

## Data Preprocessing

In [13]:
TRANSFORM_MODULE_FILE = 'data_transform.py'

In [17]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = 'status'
FEATURE_KEY = 'statement'
def transformed_name(key):
    return key + "_xf"

def preprocessing_fn(inputs):
    
    outputs = {}
    # Pastikan teks dalam bentuk DenseTensor sebelum transformasi
    text_input = inputs[FEATURE_KEY]
    # if isinstance(text_input, tf.SparseTensor):
    #     text_input = tf.sparse.to_dense(text_input, default_value="")

    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(text_input)
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    return outputs

Overwriting data_transform.py


In [18]:
# import os

# # Pastikan direktori kerja pendek
# os.environ['TFX_ROOT'] = r'C:\tfx_pipeline'
# TRANSFORM_MODULE_FILE = os.path.join(os.environ['TFX_ROOT'], 'transform_module.py')


In [16]:
transform = Transform(
    examples = example_gen.outputs['examples'],
    schema = schema_gen.outputs['schema'],
    module_file = os.path.abspath(TRANSFORM_MODULE_FILE) 
)

interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


RuntimeError: tensorflow.python.framework.errors_impl.NotFoundError: {{function_node __wrapped__SaveV2_dtypes_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to create a NewWriteableFile: pipelines\heart-attack-pipeline\Transform\transform_graph\10\.temp_path\tftransform_tmp\b4013f7243eb4e859fde57807ac85d46\variables\variables_temp/part-00000-of-00001.data-00000-of-00001.tempstate12261729868116967861 : The system cannot find the path specified.
; No such process [Op:SaveV2] [while running 'Analyze/CreateSavedModel[tf_v2_only]/CreateSavedModel']

## Pengembangan Model

In [ ]:
TRAINER_MODULE_FILE = "mental_health_trainer.py"

In [ ]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs